## Import stuff

In [12]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11975267176570203958
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


In [34]:


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


## Load training data from csv


In [3]:
features_df = pd.read_csv("train_features_fixed.csv")
labels_df = pd.read_csv("train_labels.csv")


# load all columns except the first two
X = np.array(features_df.iloc[:, 2:].values,  dtype=float)
X = X.reshape(227940//12,35*12)

#use train to validation ratio of 0.8
X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]

Y1 = np.array(labels_df.iloc[:, 1:11].values,  dtype=float)
Y1_train = Y1[:int(Y1.shape[0]*0.8)]
Y1_valid = Y1[int(Y1.shape[0]*0.8):]

Y2 = np.array(labels_df.iloc[:, 11].values,  dtype=float)
Y2_train = Y2[:int(Y2.shape[0]*0.8)]
Y2_valid = Y2[int(Y2.shape[0]*0.8):]

Y3 = np.array(labels_df.iloc[:, 12:].values,  dtype=float)
Y3_train = Y3[:int(Y3.shape[0]*0.8)]
Y3_valid = Y3[int(Y3.shape[0]*0.8):]

print("X shape: ", X.shape)
print("Y1 shape: ", Y1.shape)
print("Y2 shape: ", Y2.shape)
print("Y3 shape: ", Y3.shape)



# print(inputs_dim)
# print(outputs_dim)

# X = tf.cast(X, tf.float32)
# Y = tf.cast(Y, tf.float32)

#print(X.shape)
# random.shuffle(image_ids)
# random.shuffle(diagnostics)

X shape:  (18995, 420)
Y1 shape:  (18995, 10)
Y2 shape:  (18995,)
Y3 shape:  (18995, 4)


## TASK 1: ORDERING OF MEDICAL TEST

In [15]:
inputs_dim = X.shape[1]
outputs_dim = Y1.shape[1]

model1 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model1.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model1.fit(X_train, Y1_train, epochs=5) # , verbose=False
print("Finished training the model")

Epoch 1/5
475/475 [==============================] - 0s 668us/step - loss: 15758.9990 - mean_squared_error: 15758.9990
Epoch 2/5
475/475 [==============================] - 0s 634us/step - loss: 0.2011 - mean_squared_error: 0.2011
Epoch 3/5
475/475 [==============================] - 0s 756us/step - loss: 0.1508 - mean_squared_error: 0.1508
Epoch 4/5
475/475 [==============================] - 0s 726us/step - loss: 0.1361 - mean_squared_error: 0.1361
Epoch 5/5
475/475 [==============================] - 0s 630us/step - loss: 0.1335 - mean_squared_error: 0.1335
Finished training the model


## TASK 1: EVALUATION

In [35]:
pred = model1.predict(X_valid)
for i in range(Y1_valid.shape[1]):
    print("score for col ",i,": " , metrics.roc_auc_score(Y1_valid[:,i], pred[:,i]))



score for col  0 :  0.5
score for col  1 :  0.5
score for col  2 :  0.5
score for col  3 :  0.5
score for col  4 :  0.5
score for col  5 :  0.5
score for col  6 :  0.5
score for col  7 :  0.5
score for col  8 :  0.5
score for col  9 :  0.5


## TASK 2: SEPSIS PREDICTION

In [7]:
inputs_dim = X.shape[1]
outputs_dim = 1

model2 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model2.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model2.fit(X_train, Y2_train, epochs=5) # , verbose=False
print("Finished training the model")

Epoch 1/5
475/475 [==============================] - 0s 609us/step - loss: 676025.1875 - mean_squared_error: 676025.1875
Epoch 2/5
475/475 [==============================] - 0s 681us/step - loss: 0.0676 - mean_squared_error: 0.0676
Epoch 3/5
475/475 [==============================] - 0s 620us/step - loss: 0.0634 - mean_squared_error: 0.0634
Epoch 4/5
475/475 [==============================] - 0s 657us/step - loss: 0.0606 - mean_squared_error: 0.0606
Epoch 5/5
475/475 [==============================] - 0s 628us/step - loss: 0.0589 - mean_squared_error: 0.0589
Finished training the model


## TASK 2: EVALUATION

In [37]:
pred = model2.predict(X_valid)
print("score : " , metrics.roc_auc_score(Y1_valid, pred))

score :  0.4994300360631727


## TASK 3: KEYS VITALS SIGNS PREDICTION

In [9]:
inputs_dim = X.shape[1]
outputs_dim = Y3.shape[1]

model3 = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model3.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])

history = model3.fit(X_train, Y3_train, epochs=5) # , verbose=False
print("Finished training the model")

Epoch 1/5
475/475 [==============================] - 0s 683us/step - loss: 209730.4375 - mean_squared_error: 209730.4375
Epoch 2/5
475/475 [==============================] - 0s 642us/step - loss: 4135.9883 - mean_squared_error: 4135.98830s - loss: 4612.7314 - mean_squared_error:
Epoch 3/5
475/475 [==============================] - 0s 653us/step - loss: 2806.4231 - mean_squared_error: 2806.4231
Epoch 4/5
475/475 [==============================] - 0s 622us/step - loss: 1776.3888 - mean_squared_error: 1776.3888
Epoch 5/5
475/475 [==============================] - 0s 615us/step - loss: 1043.7279 - mean_squared_error: 1043.7279
Finished training the model


## TASK 3: EVALUATION

In [38]:
pred = model3.predict(X_valid)
for i in range(Y3_valid.shape[1]):
    print("score for col ",i,": " , 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y3_valid[i], pred[i])))



score for col  0 :  0.7012885106810924
score for col  1 :  0.6708455578812551
score for col  2 :  0.5
score for col  3 :  0.7032524440733434


## Writing results